# Deliverable 3. Create a Travel Itinerary Map.
----
1. Import the vacation CSV file. 
2. From the map above with pick 4 cities that are in close proximity (on the same continent) that the customer would travel to and create a directions layer map.
3. For the "travel_mode" use either "DRIVING", "BICYCLING", or "WALKING".
4. Take a screenshot of the route and save it. 
5. Then create a marker layer for the four cities. Each city will should have a pop-up marker that contains
    - The hotel name
    - The city
    - The country
    - The current weather description and the maximum temperature
---

## Make sure the initial dependencies and the Google API key are imported.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

In [2]:
# Configure gmaps
gmaps.configure(api_key=g_key)

## From your `Vacation_Search` folder from Deliverable 2, import the `WeatherPy_vacation.csv` file as a DataFrame named `vacation_df`

In [3]:
# Read the WeatherPy_vacation.csv into a DataFrame
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv", index_col="City_ID")

# Display sample data
vacation_df.head(10)

,City,Country,Lat,Lng,Max Temp,Current Description,Hotel Name
City_ID,,,,,,,
0,Yulara,AU,-25.2406,130.9889,76.41,overcast clouds,Desert Gardens Hotel - Ayers Rock Resort
1,Hasaki,JP,35.7333,140.8333,86.29,broken clouds,Inubosaki Kanko Hotel
2,Butaritari,KI,3.0707,172.7902,82.09,scattered clouds,Isles Sunset Lodge
3,Lorengau,PG,-2.0226,147.2712,86.05,broken clouds,Lorengau Harbourside Hotel
4,Tuy Hoa,VN,13.0833,109.3000,78.73,overcast clouds,CenDeluxe Hotel Tuy Hoa
5,Saint-Philippe,RE,-21.3585,55.7679,76.59,scattered clouds,Le four à pain
6,Hithadhoo,MV,-0.6000,73.0833,82.44,overcast clouds,Scoop Guest House
7,Lawigan,PH,10.4870,122.0354,86.72,broken clouds,The Assemblage Point Resort and Convention Hub
8,Phuket,TH,8.0000,98.2500,86.22,scattered clouds,The Surin


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [ ]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## In this step, you will set-up the pop-up markers. Review the code to create a marker layer map of the vacation search results. This code is the same as in Deliverable 2.

In [4]:
# Review the formatting template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [5]:
# Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(10.0, 35.0), zoom_level=1.6)
fig.add_layer(marker_layer)

# Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

## From the vacation search map, *choose four cities* that a customer might want to visit. They should be close together and in the same country.

In [6]:
# Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# The starting and ending city should be the same city.
vacation_start = vacation_df.loc[(vacation_df["City"]=="Rengasdengklok") & (vacation_df["Country"]=="ID")].copy() #Rengasdengklok, ID
vacation_end = vacation_df.loc[(vacation_df["City"]=="Rengasdengklok") & (vacation_df["Country"]=="ID")].copy()
vacation_stop1 = vacation_df.loc[(vacation_df["City"]=="Kawalu") & (vacation_df["Country"]=="ID")].copy() #Kawalu, ID
vacation_stop2 = vacation_df.loc[(vacation_df["City"]=="Srandakan") & (vacation_df["Country"]=="ID")].copy() #Srandakan, ID
vacation_stop3 = vacation_df.loc[(vacation_df["City"]=="Labuhan") & (vacation_df["Country"]=="ID")].copy() #Labuhan, ID

vacation_stop2

,City,Country,Lat,Lng,Max Temp,Current Description,Hotel Name
City_ID,,,,,,,
21,Srandakan,ID,-7.9386,110.2506,82.38,broken clouds,Puri Brata


## Use the `to_numpy()` function and list indexing to write code to retrieve the latitude-longitude pairs as tuples from each city DataFrame.

In [7]:
# Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0]
end = vacation_end["Lat"].to_numpy()[0], vacation_end["Lng"].to_numpy()[0]
stop1 = vacation_stop1["Lat"].to_numpy()[0], vacation_stop1["Lng"].to_numpy()[0]
stop2 = vacation_stop2["Lat"].to_numpy()[0], vacation_stop2["Lng"].to_numpy()[0]
stop3 = vacation_stop3["Lat"].to_numpy()[0], vacation_stop3["Lng"].to_numpy()[0]

start, stop1, stop2, stop3, end

((-6.1592, 107.2981),
 (-7.3817, 108.2082),
 (-7.9386, 110.2506),
 (-6.8844, 112.2051),
 (-6.1592, 107.2981))

## Use the [gmaps documentation](https://jupyter-gmaps.readthedocs.io/en/latest/tutorial.html#directions-layer) to create a directions layer map using the variables from the previous step. Where the starting and ending city are the same city, the `waypoints` are the three other cities, and the `travel_mode` is either `"DRIVING"`, `"BICYCLING"`, or `"WALKING"`.

In [8]:
# Define a new figure object
fig = gmaps.figure()

# Create a direction layer map using the start and end latitude-longitude pairs, and stop1, stop2, and stop3 as the waypoints.
# The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
vacation_itinerary = gmaps.directions_layer(start, end, waypoints=[stop1, stop2, stop3], travel_mode='DRIVING')

# Add the layer to the map
fig.add_layer(vacation_itinerary)

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))

## Use the provided `concat()` function code snippet to combine the four separate city DataFrames into one DataFrame.

In [9]:
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat(
    [
        vacation_start,
        vacation_stop1,
        vacation_stop2,
        vacation_stop3
    ],
    ignore_index = True
)

# Display sample data
itinerary_df

,City,Country,Lat,Lng,Max Temp,Current Description,Hotel Name
0,Rengasdengklok,ID,-6.1592,107.2981,89.73,overcast clouds,Tanah dan Rumah Singgah Keluarga
1,Kawalu,ID,-7.3817,108.2082,79.03,overcast clouds,West Jontor
2,Srandakan,ID,-7.9386,110.2506,82.38,broken clouds,Puri Brata
3,Labuhan,ID,-6.8844,112.2051,86.90,overcast clouds,DoubleFAG


## Refactor the code from Step 6 to create a new marker layer map of the cities on the travel route.

In [18]:
# Review the formatting template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# Get the data from each row and add it to the formatting template and store the data in a list
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]
print(hotel_info)

# Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]
print(locations)

['\n<dl>\n<dt>Hotel Name</dt><dd>Tanah dan Rumah Singgah Keluarga</dd>\n<dt>City</dt><dd>Rengasdengklok</dd>\n<dt>Country</dt><dd>ID</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 89.73 °F</dd>\n</dl>\n', '\n<dl>\n<dt>Hotel Name</dt><dd>West Jontor</dd>\n<dt>City</dt><dd>Kawalu</dd>\n<dt>Country</dt><dd>ID</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 79.03 °F</dd>\n</dl>\n', '\n<dl>\n<dt>Hotel Name</dt><dd>Puri Brata</dd>\n<dt>City</dt><dd>Srandakan</dd>\n<dt>Country</dt><dd>ID</dd>\n<dt>Current Weather</dt><dd>broken clouds and 82.38 °F</dd>\n</dl>\n', '\n<dl>\n<dt>Hotel Name</dt><dd>DoubleFAG</dd>\n<dt>City</dt><dd>Labuhan</dd>\n<dt>Country</dt><dd>ID</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 86.9 °F</dd>\n</dl>\n']
      Lat       Lng
0 -6.1592  107.2981
1 -7.3817  108.2082
2 -7.9386  110.2506
3 -6.8844  112.2051


In [19]:
# Define a new figure object
fig = gmaps.figure()

# Add a marker layer for each city to the map.
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create a direction layer map
# will hide the directions markers so the hotel markers show instead
vacation_itinerary = gmaps.directions_layer(start, end, waypoints=[stop1, stop2, stop3], travel_mode='DRIVING',\
                                           show_markers=False)

# Add the layers to the map
fig.add_layer(marker_layer)
fig.add_layer(vacation_itinerary)

# Display the map
fig

Figure(layout=FigureLayout(height='420px'))